In [1]:
import warnings
warnings.simplefilter(action='ignore', category=UserWarning)

import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots
import gzip
from datetime import datetime, timedelta
from statistics import mean, median
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
import seaborn as sns
import matplotlib.pyplot as plt

import tensorflow
import tensorflow.keras as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Dense, LeakyReLU, BatchNormalization, ReLU, LSTM, Conv1D, Conv2D
from tensorflow.keras.activations import sigmoid, tanh
from tensorflow.keras.utils import to_categorical

from sklearn.preprocessing import MinMaxScaler

from tqdm import tqdm
import csv
import random

from sklearn.metrics import accuracy_score as accuracy
from sklearn.metrics import precision_score as precision
from sklearn.metrics import recall_score as recall
from sklearn.metrics import f1_score as f1

In [2]:
def retrieve_data(varname, filename):
    df = pd.read_csv(filename, index_col=0)
    df["Date"] = pd.to_datetime(df["Date"])
    return df

def create_classification_data(df, lookback):
    rows = []
    columns = ['Date', 'US30_relative_change_perc_1'] # Date and SP500_relative_change_perc_1 from t-0 are added first as target variables 
    
    # create column names based on original with the addition of t-i where i is lookback
    for i in range(1, lookback + 1): # starts at 1 since we do not want t-0 variables apart from 'Date' and 'SP500_relative_change_perc_1'
        new_columns = df.columns.tolist()[1:] # starts at 1 to exclude 'Date' column
        for x in range(len(new_columns)):
            new_columns[x] = new_columns[x] + "_t-" + str(i)
        columns = columns + new_columns
    
    # create lookback data
    for i, row in enumerate(df.iterrows()):
        if i > lookback: # lookback cannot be determined for earlier rows
            new_row = [row[1][0], row[1][1]] # add target 'Date' and 'SP500_relative_change_perc_1 '
            for x in range(1, lookback + 1): # starts at 1 since we do not want t-0 variables apart from 'Date' and 'SP500_relative_change_perc_1'
                add_row = df.iloc[i - x].tolist()[1:] # starts at 1 to exclude 'Date' column
                new_row = new_row + add_row
            rows.append(new_row)
    df2 = pd.DataFrame(rows)
    df2.columns = columns
    return df2

def create_train_val_test(df, year_val, year_test, perc_train=None):
    if perc_train == None:
        # assumes years_train < year_val < year_test
        df["Date"] = pd.to_datetime(df["Date"])
        
        val = df[df['Date'].dt.year == year_val]
        test = df[df['Date'].dt.year == year_test]
        train = df[df['Date'].dt.year < year_val]
    else:
        train = df.head(round(len(df) * perc_train))
        val = df.tail(len(df) - len(train))
        test = val.tail(round(0.5 * len(val)))
        val = df.head(len(val) - len(test))
    y_train = train['US30_relative_change_perc_1']
    x_train = train.drop(['US30_relative_change_perc_1'], axis=1)
    
    y_val = val['US30_relative_change_perc_1']
    x_val = val.drop(['US30_relative_change_perc_1'], axis=1)
    
    y_test = test['US30_relative_change_perc_1']
    x_test = test.drop(['US30_relative_change_perc_1'], axis=1)
    
    return x_train, y_train, x_val, y_val, x_test, y_test

def scale_data(x):
    standard_scaler = MinMaxScaler()
    x = x.drop(["Date"], axis=1)
    x_scaled = pd.DataFrame(standard_scaler.fit_transform(x), columns=x.columns)
    return x_scaled

In [3]:
def label_data(y):
    positives = []
    negatives = []
    y = list(y)
    for dev in y:
        if dev >= 0:
            positives.append(dev)
        else:
            negatives.append(dev)
    med_pos = median(positives)
    med_neg = median(negatives)
    
    labels = []
    for dev in y:
        if dev >= 0:
            if dev >= med_pos:
                labels.append(2)
            else:
                labels.append(1)
        else:
            if dev <= med_neg:
                labels.append(-2)
            else:
                labels.append(-1)
    return labels

In [4]:
def determine_transformed_distribution(y):
    counts = [0, 0, 0, 0]
    for i in range(len(y)):
        counts[np.argmax(y[i])] += 1
    print(counts)
    
def one_hot_encode(y):
    one_hot = []
    for i in y:
        if i == -2:
            one_hot.append([1,0,0,0])
        elif i == -1:
            one_hot.append([0,1,0,0])
        elif i == 1:
            one_hot.append([0,0,1,0])
        elif i == 2:
            one_hot.append([0,0,0,1])
    return np.asarray(one_hot)

In [5]:
class LSTM_model(object):
    def __init__(self, x, activation_functions, batch_sizes):        
        self.activation_function = activation_functions[x[0]]
        
        self.lstm1 = x[1]
        self.lstm2 = x[2]
        
        self.dense1 = x[3]
        self.dense2 = x[4]

        self.dropout1 = x[5]
        self.dropout2 = x[6]
        
        self.epochs = x[7]
        self.batch_size = batch_sizes[x[8]]
        
        self.lookback = x[9]
        
    def fit(self):
        global df

        val_year = 2018
        test_year = 2019
        df_class = create_classification_data(df, self.lookback)
        x_train, y_train, x_val, y_val, x_test, y_test = create_train_val_test(df_class, val_year, test_year)


        y_train = label_data(y_train)
        y_val = label_data(y_val)
        y_test = label_data(y_test)

        train_date = x_train[['Date']]
        x_train = x_train.drop(['Date'], axis=1)
        val_date = x_val[['Date']]
        x_val = x_val.drop(['Date'], axis=1)
        test_date = x_test[['Date']]
        x_test = x_test.drop(['Date'], axis=1)

        x_train = np.asarray(x_train)
        x_val = np.asarray(x_val)
        x_test = np.asarray(x_test)
        y_train = np.asarray(y_train)
        y_val = np.asarray(y_val)
        y_test = np.asarray(y_test)

        x_train = x_train.reshape((x_train.shape[0], 1, x_train.shape[1]))
        x_val = x_val.reshape((x_val.shape[0], 1, x_val.shape[1]))
        x_test = x_test.reshape((x_test.shape[0], 1, x_test.shape[1]))

        y_train = one_hot_encode(y_train)
        y_val = one_hot_encode(y_val)
        y_test = one_hot_encode(y_test)
#         determine_transformed_distribution(y_train)
#         determine_transformed_distribution(y_val)
#         determine_transformed_distribution(y_test)

        y_train = y_train.reshape((y_train.shape[0], 1, y_train.shape[1]))
        y_val = y_val.reshape((y_val.shape[0], 1, y_val.shape[1]))
        y_test = y_test.reshape((y_test.shape[0], 1, y_test.shape[1]))

#         print(x_train.shape, y_train.shape)
#         print(x_val.shape, y_val.shape)
#         print(x_test.shape, y_test.shape)
        
        model = Sequential()
        model.add(LSTM(self.lstm1, dropout=self.dropout1, input_shape=(x_train.shape[1], x_train.shape[2]), return_sequences=True))
        if self.lstm2 > 0:
            model.add(LSTM(self.lstm2, dropout=self.dropout2, return_sequences=True))
        if self.dense1 > 0:
            model.add(Dense(self.dense1, activation=self.activation_function))
        if self.dense2 > 0:
            model.add(Dense(self.dense2, activation=self.activation_function))
        model.add(Dense(4, activation='softmax'))
        
        model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=["acc"])
        history = model.fit(x_train, y_train, epochs=self.epochs, batch_size=self.batch_size, verbose=0, validation_data=(x_val, y_val), shuffle=False)
        
        val_loss = mean(history.history['val_acc'][-5:])
#         val_loss = history.history['val_acc'][-1]
        return val_loss

In [6]:
def calculate_fitness(val_loss):
    fitness = val_loss
    return fitness

In [7]:
class EA(object):
    def __init__(self, population_size, activation_functions, batch_sizes):
        self.population_size = population_size
        self.a = 0.2
        self.activation_functions = activation_functions
        self.batch_sizes = batch_sizes
        
    def evaluate(self, x):
        """
        include in fitness function
            relative difference between train_loss and val_loss (smaller is better)
            number of layers (smaller is better)
            bottleneck size (smaller is better)
            val_loss (smaller is better)
        
        """
        lstm = LSTM_model(x, self.activation_functions, self.batch_sizes)
        val_loss = lstm.fit()
        fitness = calculate_fitness(val_loss)
        return fitness
    
    def select_triple(self, candidate, population):
        # select three random instances for differential evolution
        x1, x2, x3 = np.random.choice(range(len(population))), np.random.choice(range(len(population))), np.random.choice(range(len(population)))
        while candidate == x1 or candidate == x2 or candidate == x3 or x1 == x2 or x2 == x3 or x1 == x3:
            # keep selecting new ones until candidate != x1 != x2 != x3
            x1, x2, x3 = np.random.choice(range(len(population))), np.random.choice(range(len(population))), np.random.choice(range(len(population)))
        return population[x1], population[x2], population[x3]
    
    def mutate(self, x1, x2, x3):
        mutated = x1 + (self.a * (x3 - x2))
#         print(f"mutated {mutated}")
        # activation function
        mutated[0] = round(mutated[0])
        mutated[0] = min(mutated[0], len(self.activation_functions) - 1)
        mutated[0] = max(0, mutated[0])
        
        # lstm layer 1
        mutated[1] = round(mutated[1])
        mutated[1] = max(1, mutated[1]) # must be at least one

        # lstm layer 2
        mutated[2] = round(mutated[2])
        mutated[2] = max(0, mutated[2])

        # dense layer 1
        mutated[3] = round(mutated[3])
        mutated[3] = max(4, mutated[3])
        
        # dense layer 2
        mutated[4] = round(mutated[4])
        mutated[4] = max(4, mutated[4])

        # dropout lstm layer 1
        mutated[5] = max(0.05, mutated[5])
        mutated[5] = min(0.95, mutated[5])
        mutated[5] = round(mutated[5],2)
        
        # dropout lstm layer 2
        mutated[6] = max(0.05, mutated[6])
        mutated[6] = min(0.95, mutated[6])
        mutated[6] = round(mutated[6],2)
        
        # epochs
        mutated[7] = round(mutated[7])
        mutated[7] = max(6, mutated[7])

        # batch size
        mutated[8] = round(mutated[8])
        mutated[8] = min(mutated[8], len(self.batch_sizes) - 1)
        mutated[8] = max(0, mutated[8])
        
        mutated[9] = round(mutated[9])
        mutated[9] = max(1, mutated[9])
        
        return mutated
        
    def recombine(self, candidate, mutation):
        for i in range(candidate.shape[0]):
            prob = np.random.randint(0, 2)
            if prob == 1:
                candidate[i] = mutation[i]
        return candidate

    def select(self, x_new, f_new, x_old, f_old):
        x_cat = np.concatenate([x_new, x_old], 0)
        f_cat = np.concatenate([f_new, f_old])
        ind = np.argsort(f_cat)
        x = x_cat[ind]
        f = f_cat[ind]
        return x[-self.population_size:], f[-self.population_size:]
    
    def step(self, x_old, f_old):
        x = np.copy(x_old)
        f = np.copy(f_old)
        for i in tqdm(range(self.population_size), total=self.population_size):
            # choose candidate
            candidate = x[i]
            # select 3 instances for differential evolution
            x1, x2, x3 = self.select_triple(i, x)
            # mutate 3 instances
            mutated_triple = self.mutate(x1, x2, x3)
            # recombine candidate with mutation
            candidate = self.recombine(candidate, mutated_triple)
            x[i] = candidate
            # evaluate candidate solution
            f_candidate = self.evaluate(candidate)
            f[i] = f_candidate
        # select survivors
        x, f = self.select(x, f, x_old, f_old)
        return x, f

In [8]:
def init_population(population_size, activation_functions, batch_sizes):
    # generate initial population
    population = []
    print("Creating initial population...")
    for i in tqdm(range(population_size), total=population_size):
        activation_function = random.randint(0, len(activation_functions) - 1)
        
        lookback = random.randint(1, 20)
        
        variables = lookback * 18
        
        lstm1 = random.randint(1, 1.5 * variables)
        lstm2 = random.randint(0, 1.5 * variables)
        dense1 = random.randint(4, 1.5 * variables)
        dense2 = random.randint(4, 1.5 * variables)
        
        dropout1 = round(random.uniform(0.05, 0.95),2)
        dropout2 = round(random.uniform(0.05, 0.95),2)

        epochs = random.randint(10, 500)
        
        batch_size = random.randint(0, len(batch_sizes) - 1)
    
        population.append(np.asarray([activation_function, lstm1, lstm2, dense1, dense2, dropout1, dropout2, epochs, batch_size, lookback], dtype='object'))
    print("Initial population ready")
    return np.asarray(population)

def evaluate_init_population(ea, x):
    # evaluate initial population
    f = []
    print("Evaluating initial population...")
    for i in tqdm(range(x.shape[0]), total=x.shape[0]):
        instance = x[i]
        f.append(ea.evaluate(instance))
    print("Evaluation initial population completed")
    return np.asarray(f)

def print_best(x, activation_functions, batch_sizes, fitness):
    print(f"\nMost suitable parameters -- Accuracy of {fitness}:")
    print(f"\tActivation function:           \t{activation_functions[x[0]]}")
    print(f"\tLSTM nodes layer 1:            \t{x[1]}")
    print(f"\tLSTM nodes layer 2:            \t{x[2]}")
    print(f"\tDense nodes layer 1:           \t{x[3]}")
    print(f"\tDense nodes layer 2:           \t{x[4]}")
    print(f"\tDropout LSTM layer 1:          \t{x[5]}")
    print(f"\tDropout LSTM layer 2:          \t{x[6]}")
    print(f"\tEpochs trained:                \t{x[7]}")
    print(f"\tBatch Size:                    \t{batch_sizes[x[8]]}")
    print(f"\tLookback:                      \t{x[9]}")

def plot_convergence(f_best):
    fig1 = make_subplots(rows=1, cols=1, specs=[[{'type':'xy'}]])
    
    x_values = []
    for i in range(len(f_best)):
        x_values.append(i)
    fig1.add_trace(go.Scatter(x=x_values, y=f_best, mode="lines"), row=1, col=1)

    fig1.update_layout(
        title = f'Validation Accuracy Over Autoencoder Tuning Generations', 
        xaxis1 = dict(title_text = 'Generation'),
        yaxis1 = dict(title_text = "Validation Accuracy")
    )
    fig1.write_image("Plots/opt lstm 2 us30.png")
    fig1.show()

def validate_best(x, ea):
    print("\nValidating solution...")
    ea.evaluate(x)
    print("Solution validated")

In [9]:
df = retrieve_data("SP500", "Dataset v3/US30_reduced_data_20220425.csv")

population_size = 30
generations = 30
activation_functions = ['sigmoid', 'tanh']
batch_sizes = [64, 128, 256]

ea = EA(population_size, activation_functions, batch_sizes)
x = init_population(population_size, activation_functions, batch_sizes)
f = evaluate_init_population(ea, x)

populations = []
populations.append(x)
f_best = [f.max()]

start_time = datetime.now()

print("--> STARTING EVOLUTION")
early_stop = 0
for i in range(generations):
    print(f'Generation: {i}\tBest fitness: {f.max()}')
    x, f = ea.step(x, f)
    print(x)
    populations.append(x)

    if f.max() > f_best[-1]:
        f_best.append(f.max())
        early_stop = 0
    else:
        f_best.append(f_best[-1])
        early_stop += 1
    if early_stop == 3:
        print("Early stop triggered at generation {i} after not improving fitness for three generations")
        break
print("--> EVOLUTION FINISHED")

end_time = datetime.now()
evolution_time = end_time - start_time
evolution_time_seconds = evolution_time.total_seconds()
print(f"\nElapsed time in minutes: {evolution_time_seconds/60}")

print(f)
print(f.min())
index_best_parameters = np.where(f == f.max())[0][0]
print(index_best_parameters)
print_best(x[index_best_parameters], activation_functions, batch_sizes, f.max())
validate_best(x[index_best_parameters], ea)
plot_convergence(f_best)

  0%|                                                                                           | 0/30 [00:00<?, ?it/s]

Creating initial population...
Initial population ready
Evaluating initial population...


2022-04-29 13:20:16.684393: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-04-29 13:20:16.684846: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-04-29 13:20:17.378634: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
  0%|                                                                                           | 0/30 [00:00<?, ?it/s]

Evaluation initial population completed
--> STARTING EVOLUTION
Generation: 0	Best fitness: 0.35936256051063536


  0%|                                                                                           | 0/30 [00:00<?, ?it/s]

...

[[0 161 77 47 25 0.87 0.68 126 2 4]
 [1 49 10 395 38 0.91 0.41 490 1 5]
 [0 103 77 28 25 0.87 0.68 126 1 4]
 [1 74 151 155 47 0.78 0.7 262 2 5]
 [1 93 117 90 69 0.78 0.81 472 0 5]
 [0 177 115 36 82 0.84 0.83 219 2 5]
 [1 156 117 162 4 0.84 0.52 472 2 6]
 [1 103 151 268 44 0.95 0.68 464 2 4]
 [1 17 179 298 4 0.4 0.74 425 1 5]
 [1 160 96 52 81 0.88 0.85 554 2 6]
 [1 177 130 96 26 0.84 0.83 455 0 5]
 [1 24 204 269 479 0.84 0.48 278 0 18]
 [1 80 167 92 53 0.77 0.78 171 2 12]
 [1 24 161 98 479 0.77 0.78 278 0 12]
 [1 93 126 90 69 0.88 0.71 335 0 5]
 [0 314 126 59 75 0.9 0.53 259 2 5]
 [0 73 126 159 34 0.8 0.68 126 2 6]
 [1 47 20 456 38 0.93 0.32 454 1 5]
 [1 76 78 94 57 0.9 0.84 489 0 5]
 [1 135 113 61 72 0.89 0.78 34 2 6]
 [1 76 135 94 57 0.9 0.84 489 0 5]
 [1 156 117 121 169 0.56 0.81 472 2 6]
 [1 93 126 90 69 0.78 0.57 335 0 5]
 [1 76 132 94 50 0.86 0.84 385 0 6]
 [1 80 3 92 58 0.88 0.68 487 2 4]
 [1 46 204 134 26 0.84 0.48 278 2 6]
 [1 40 234 104 110 0.83 0.48 262 2 5]
 [1 93 97 90 66 0

  0%|                                                                                           | 0/30 [00:00<?, ?it/s]

[[1 177 115 182 82 0.84 0.47 219 2 6]
 [1 160 96 52 81 0.88 0.85 554 2 6]
 [1 139 117 162 67 0.84 0.52 6 2 6]
 [1 177 130 96 26 0.84 0.83 455 0 5]
 [1 24 204 269 479 0.84 0.48 278 0 18]
 [1 18 234 104 465 0.79 0.48 217 2 13]
 [1 80 167 92 53 0.77 0.78 171 2 12]
 [1 93 126 90 69 0.88 0.71 335 0 5]
 [1 24 161 98 479 0.77 0.78 278 0 12]
 [1 160 219 123 81 0.88 0.85 346 2 6]
 [0 73 126 159 34 0.8 0.68 126 2 6]
 [1 76 107 94 67 0.79 0.84 489 0 5]
 [0 314 126 59 75 0.9 0.53 259 2 5]
 [1 76 78 94 57 0.9 0.84 489 0 5]
 [1 135 113 61 72 0.89 0.78 34 2 6]
 [1 47 20 456 38 0.93 0.32 454 1 5]
 [1 76 135 94 57 0.9 0.84 489 0 5]
 [1 156 117 121 169 0.56 0.81 472 2 6]
 [1 161 77 184 25 0.83 0.45 126 2 4]
 [1 93 126 102 69 0.88 0.71 335 2 5]
 [0 93 97 96 66 0.78 0.81 292 2 5]
 [1 93 126 90 69 0.78 0.57 335 0 5]
 [1 76 132 94 50 0.86 0.84 385 0 6]
 [1 80 3 92 58 0.88 0.68 487 2 4]
 [1 46 204 134 26 0.84 0.48 278 2 6]
 [1 40 234 104 110 0.83 0.48 262 2 5]
 [1 93 97 90 66 0.82 0.81 292 0 5]
 [0 76 152 17

 83%|█████████████████████████████████████████████████████████████████▊             | 25/30 [1:12:14<14:26, 173.39s/it]


KeyboardInterrupt: 

In [ ]:
plot_convergence(f_best)